# VAE Auto Encoder Network


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [7]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import os
import random

import tensorflow as tf 
import tensorflow.keras
import keras 
from keras import layers
from keras import backend as K
from keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [8]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.set_random_seed(7)

AttributeError: ignored

In [9]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#디렉토리 변경
os.chdir('drive/My Drive/Colab Notebooks/Dacon/AIFrenz_Season2')


In [6]:
pwd

'/content'

### VAE 인코더 네트워크


In [0]:
img_shape = (40,40,10)
batch_size = 16
latent_dim = 2 # 잠재 공간의 차원: 2D 평면


In [0]:
input_img = keras.Input(shape=img_shape)


In [0]:
x = layers.Conv2D(32, 3 , padding ='same', activation = 'relu')(input_img)
x = layers.Conv2D(64, 3 , padding ='same', activation = 'relu', strides = (2,2))(x)
x = layers.Conv2D(64, 3 , padding ='same', activation = 'relu')(input_img)
x = layers.Conv2D(64, 3 , padding ='same', activation = 'relu')(input_img)

shape_before_flattening = K.int_shape(x)

In [0]:
x = layers.Flatten()(x)
x = layers.Dense(32, activation= 'relu')(x)


In [0]:
# 입력 이미지 2개의 파리미터로 인코딩
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

### 잠재 공간 샘플링 함수

In [0]:
def sampling(args):
  z_mena, z_log_var = args
  epsilon = K.random_normal(shape = (K.shape(z_mean)[0], latent_dim), mean = 0., stddev = 1.)
  return z_mean + K.exp(0.5 * z_log_var) * epsilon

z= layers.Lambda(sampling)([z_mean, z_log_var])

### 잠재 공간 포인트를 이미지로 매핑하는 VAE 디코더 네트워크


In [16]:
decoder_input = layers.Input(K.int_shape(z)[1:]) # Input에 z를 주입

#입력을 업샘플링
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation= 'relu')(decoder_input)

#인코더 모델의 마지막 Flatten 층 직전의 특성 맵과 같은 크기를 가진 특성 맵으로 z의 크기를 바꿈
x = layers.Reshape(shape_before_flattening[1:])(x)

#Conv2DTranspose층 Conv2D층을 상용하여 z를 원본 입력 이미지와 같은 크기의 특성 맵으로 디코딩
x = layers.Conv2DTranspose(32, 3, padding = 'same', activation='relu', strides=(2,2))(x)
x = layers.Conv2D(1, 3, padding='same', activation = 'sigmoid')(x)

# decoder_input을 디코딩된 이미지로 변환하는 디코더 모델의 객체를 만듬
decoder = Model(decoder_input, x)

# 모델에 z를 주입하면 디코딩된 z를 출력
z_decoded = decoder(z)


AttributeError: ignored

### VAE 손실을 계산하기 위해 직접 만든 층


In [0]:
class CustomVariationalLayer(keras.layers.Layer):

  def vae_loss(self, x, z_decoded):
    x = K.flatten(x)
    z_decoded = K.flatten(z_decoded)
    xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
    kl_loss = -5e-4 * K.mean(
        1 + z_log_var - K.exp(z_log_var), axis = -1)
    return K.mean(xent_loss, kl_loss)

# call 메서드가 있는 층을 구현
  def call(self, inputs):
    x = inputs[0]
    z_decoded = inputs[1]
    loss = self.vae_loss(x, z_decoded)
    self.add_loss(loss, inputs = inputs)
    return x # 이 출력을 사용하지는 않지만 층은 무언가를 반환해야 함




In [11]:
#입력과 디코딩된 출력으로 이 층을 호출하여 모델의 최종 출력을 얻는다
y = CustomVariationalLayer()([input_img, z_decoded])

NameError: ignored

### VAE 훈련하기

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split , KFold

In [0]:
WIDTH = 40
HEIGHT = 40
CHANNEL = 9
IMAGE_SIZE = [WIDTH,HEIGHT,CHANNEL]
numOutputClass  = 1

file_path = './data/'
output_path = 'ckpt/output'
pred_path = 'ckpt/prediction'

NUM_TRAIN = 76345
NUM_TEST = 2416
BATCH_SIZE = 512
TOTAL_BATCH = int(NUM_TRAIN/BATCH_SIZE)
EPOCHS = 5 

In [0]:
path = './Data/' 
#train_data = np.load(path+'train_imageDataGenerator.npy')

### Data 정규화


In [0]:
train_images = np.load(path+'./train_images.npy')
train_labels = np.load(path+'./train_labels.npy')



In [0]:
train_data = np.concatenate((train_images, train_labels), axis=3)
max_val = train_data.max(axis=(0, 1, 2))
min_val = train_data.min(axis=(0, 1, 2))
train_data = (train_data - min_val) / (max_val - min_val)